# Testing Queries

Note: For now, we are lucky that the imports work, because we are using relative imports, and at one point, it will cause errors.

## IMPORTS

In [1]:
import os
import sys
import random
import requests
import argparse
from typing import List, Set

import numpy as np
import pandas as pd

In [2]:
from db.yagodb import YagoDB
from db.constants.main import YAGO_ALL_ENTITY_COUNT, YAGO_FACTS_ENTITY_COUNT
from db.functions.entity import get_random_entities_query

In [3]:
from kg.query import get_triples_multiple_subjects_query, get_description_multiple_entities_query, query_kg, get_triples_from_response

In [4]:
from utils.constants import YAGO_ENTITY_STORE_DB_PATH, YAGO_PREFIXES_PATH, YAGO_ENDPOINT_URL
from utils.prefix import get_prefixes, get_url_from_prefix_and_id
from utils.random_walk2 import SPARQL_COLUMNS_DICT, RandomWalk2

### CONSTANTS

### FUNCTIONS

### Experiment Single Walks

In [5]:
yago_db = YagoDB(YAGO_ENTITY_STORE_DB_PATH)

In [6]:
random_walk_2 = RandomWalk2(yago_db)

In [7]:
entities_df = random_walk_2.random_walk_description_batch(num_of_entities=100, depth=3)

Length of entity_list:  100
Length of entity_list:  100
Length of triples for entity0:  <class 'pandas.core.frame.DataFrame'> 235
Length of entities with valid neighbors:  17
Length of entity_list:  17
Length of entity_list:  17
Length of triples for entity1:  <class 'pandas.core.frame.DataFrame'> 5293
Length of entities with valid neighbors:  16
Length of entity_list:  11


In [11]:
entities_df

,entity0,description0,predicate1,entity1,description1,predicate2,entity2,description2
0,http://yago-knowledge.org/resource/Bangladeshi...,international cricket tour,http://schema.org/location,http://yago-knowledge.org/resource/Pakistan,sovereign state in South Asia,http://yago-knowledge.org/resource/neighbors,http://yago-knowledge.org/resource/Afghanistan,country in Central and South Asia
1,http://yago-knowledge.org/resource/Q29823478,NaN,None,None,NaN,None,None,NaN
2,http://yago-knowledge.org/resource/Michael_Lo_...,NaN,None,None,NaN,None,None,NaN
3,http://yago-knowledge.org/resource/School_At_1...,NaN,None,None,NaN,None,None,NaN
4,http://yago-knowledge.org/resource/Tramlaan_Q1...,NaN,None,None,NaN,None,None,NaN
...,...,...,...,...,...,...,...,...
95,http://yago-knowledge.org/resource/2Mass_J0525...,NaN,None,None,NaN,None,None,NaN
96,http://yago-knowledge.org/resource/Xandente_Q1...,NaN,None,None,NaN,None,None,NaN
97,http://yago-knowledge.org/resource/A_U002E__Fe...,NaN,None,None,NaN,None,None,NaN
98,http://yago-knowledge.org/resource/Retropharyn...,NaN,None,None,NaN,None,None,NaN


In [50]:
entities_df["description1"].isna().sum()

np.int64(85)

In [8]:
entities_df.iloc[25]["entity0"]

'http://yago-knowledge.org/resource/Sdss_J093110_U002E_75_U002B_135031_U002E_3_Q78051022'

In [10]:
entities_df.head(10).to_csv("walks.csv", index=False)